In [2]:
import os
from pathlib import Path
import sys
sys.path.append(str(Path.cwd().parent))

In [3]:
from src.utils.dirutils import get_data_dir, get_models_dir
import pandas as pd

In [6]:
df = pd.read_csv(get_data_dir() / "raw" / "artgraph_captions.csv")
df = df[df["human"] == "y"]

with open(get_data_dir() / "raw" / "splits" / "test.txt", "r") as f:
    test_images = f.read().splitlines()

test_images = set(test_images)

In [7]:
# copy all images in df in a new folder
from shutil import copyfile
import json

ds = []

os.makedirs("images_human", exist_ok=True)
for i, row in df.iterrows():
    if not row["image"] in test_images:
        continue
    image_path = get_data_dir() / "raw" / "images" / row["image"]
    new_image_path = f"images_human/{row['image']}"
    d = {
        "image": row["image"],
        "caption": ""
    }
    copyfile(image_path, new_image_path)

    ds.append(d)

with open("images_human/captions_human_open_flamingo.json", "w") as f:
    json.dump(ds, f, indent=4)

In [8]:
# from df take images that are not in test.txt
df = df[~df["image"].isin(test_images)]

In [15]:
df.iloc[25]["caption"], df.iloc[25]["image"]

('The artwork depicts a dramatic seascape with waves crashing against a ship in the distance in the stormy sea.',
 'ivan-aivazovsky_shipwreck-1854.jpg')

In [16]:
# order elements in images_human/captions_human_open_flamingo.json alphabetically
with open("images_human/captions_human_open_flamingo.json", "r") as f:
    ds = json.load(f)

ds = sorted(ds, key=lambda x: x["image"])

with open("images_human/captions_human_open_flamingo.json", "w") as f:
    json.dump(ds, f, indent=4)